In [1]:
import json
from konlpy.tag import Kkma
from konlpy.utils import pprint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC,SVR
from sklearn.metrics import accuracy_score,mean_squared_error,roc_auc_score
import numpy as np
from sklearn.linear_model import LogisticRegression,SGDClassifier

In [2]:
REVIEWS = "cs_reviews"
POS = "pos"

In [3]:
f = open("../reviews_in_one_txt/" + REVIEWS + ".txt","r",encoding="utf-8").readlines()

In [4]:
g = open("../reviews_in_one_txt/" + REVIEWS + "_" + POS + ".txt","r",encoding="utf-8").readlines()

In [5]:
# kkma = Kkma()
# with open("../reviews_in_one_txt/" + REVIEWS+"_nouns.txt","w",encoding="utf-8") as out:
#     for line in f:
#         course = json.loads(line)
#         nouns = kkma.nouns(course["review_text"])
# #         print(nouns)
#         out.write(" ".join(nouns)+"\n")
        

In [121]:
count = {}
for i in range(len(f)):
    course = json.loads(f[i])
    if course["강의"] != "?" and len(g[i])>1:
        if course["강의"] not in count:
            count[course["강의"]] = 0
        count[course["강의"]] += 1
upper_limit = min(count.values())
print(upper_limit)

37


In [44]:
concat_nouns_list = []
count = {}
mp = {"A":5,"B":4,"C":3,"D":2,"F":1}
# mp = {"A":1,"B":1,"C":0,"D":0,"F":0}

labels = []
for i in range(len(f)):
    line = f[i]
    course = json.loads(line)
    if course["강의"] != "?" and len(g[i])>1:
        concat_nouns = g[i]
        concat_nouns_list.append(concat_nouns)
        if count[course["강의"]] < upper_limit:
            labels.append(int(mp[course["강의"]]))
            count[course["강의"]] += 1

In [45]:
labels = np.array(labels)
concat_nouns_list = np.array(concat_nouns_list)
data_train, data_test, y_train, y_test = train_test_split(concat_nouns_list,
                                    labels,test_size=0.1,random_state=42)

In [46]:
len(data_train)

448

In [47]:
len(y_train)

448

In [48]:
list(data_train)[:2]

['(성적,NNG) (은,JX) (잘,MAG) (주,VV) (시,EPH) (는,ETD) (거,NNB) (같,VA) (습니다,EFN) (.,SF) (다만,MAG) (퀴즈,NNG) (+,SW) (플젝,UN) (비율,NNG) ((,SS) (퀴즈,NNG) (는,JX) (쉽,VA) (ㄴ,ETD) (거,NNB) (2,NR) (번,NNM) (보,VV) (아서,ECD) (의미,NNG) (없,VA) (음,ETN) (),SS) (이,MDT) (50,NR) (%,SW) (로,JKM) (매우,MAG) (높,VA) (으니,ECD) (플젝,UN) (점수,NNG) (에서,JKM) (까먹,VV) (지,ECD) (않,VXV) (도록,ECD) (주의,NNG) (하,XSV) (세요,EFN) (.,SF) (수업,NNG) (에서,JKM) (중요,NNG) (하,XSV) (ㄴ,ETD) (개념,NNG) (이,VCP) (다,ECS) (싶,VXV) (으면,ECD) (한국어,NNG) (로,JKM) (보충,NNG) (설명도,NNG) (많이,MAG) (하,VV) (어,ECS) (주,VXV) (시,EPH) (고,ECE) (좋,VA) (습니다,EFN) (.,SF) (로드,NNM) (도,JX) (전산학,NNG) (부의,NNG) (다른,MDT) (4,NR) (학점,NNG) (과목,NNG) (들,XSN) (에,JKM) (비하,VV) (어서,ECD) (매우,MAG) (넣,VV) (ㄹ,ETD) (널,NNG) (하,XSV) (ㄴ,ETD) (편,NNB) (이,VCP) (ㅂ니다,EFN) (.,SF) (플젝,UN) (이,JKS) (7,NR) (개,NNM) (이,VCP) (지만,ECE) (대부분,NNG) (쉽,VA) (게,ECD) (하,VV) (ㄹ,ETD) (수,NNB) (있,VV) (고,ECE) (랩,NNG) (도,JX) (2,NR) (번,NNM) ((,SS) (필,NNG) (참,MAG) (아니,VV) (ㅁ,ETN) (),SS) (밖에,JX) (없,VA) (었,EPT) (어요,EFN) (.,SF)\n',
 '(중간고사,NNG) (

In [49]:
y_train[:2]

array([5, 5])

In [50]:
def parse_long_str(s):
    s = s.split(',')
    s = [x.strip() for x in s]
    return s
good_lecture = parse_long_str("사랑, 천사, 친절, 유명, 짱, 참여, 최고, 만족, 유익, 열정적, 갓, 답변, 명강의")
bad_lecture = parse_long_str("집중, 혼자, 부족, 자습, 당황, 최악, 인생, 불편, 복잡, 불만, 악명, 비추, 난이(도)")
#todo try weighted average of w2v
scores = {}
def add_to_dict(l,score):
    global scores
    print("Added " + str(len(l)))
    for i in range(len(l)):
        scores[l[i]] = score
add_to_dict(good_lecture,1.0)
add_to_dict(bad_lecture,-1.0)

Added 13
Added 13


In [51]:
def enlarge_train_data(data_train,y_train):
    x = list(data_train)
    y = list(y_train)
    for k,v in scores.items():
#         x.append(str(k)+"\n")
        x.append("(%s,NNG)" % (str(k))+"\n")
        if v > 0:
            y.append(int(mp['A']))
        else:
            y.append(int(mp['F']))
    
    return np.array(x),np.array(y)

data_train,y_train = enlarge_train_data(data_train,y_train)

In [52]:
len(data_train)

474

In [53]:
len(y_train)

474

In [106]:
vectorizer = TfidfVectorizer(max_features=200000, ngram_range=(3,5))

In [107]:
vectirizer.fit(..)
X_train = vectorizer.transform(data_train)
# X_train = vectorizer.fit_transform(data_train)
X_test = vectorizer.transform(data_test)

In [108]:
print(X_train.shape)

(474, 179757)


In [109]:
print(X_test.shape)

(50, 179757)


In [110]:
clf = LinearSVC()
clf.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [111]:
predictions = clf.predict(X_test)
accuracy_score(y_test,predictions)

0.6

In [112]:
mean_squared_error(y_test,predictions)

2.24

In [113]:
predictions

array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5])

In [100]:
y_test

array([5, 2, 5, 5, 5, 5, 4, 4, 5, 4, 2, 4, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 5, 2, 5, 2, 3, 4, 5, 3, 5, 3, 5, 5, 1, 5, 5, 5, 5, 5, 4, 5,
       1, 4, 3, 5, 4, 3])

In [101]:
for i in range(len(predictions)):
    if predictions[i]!= y_test[i]:
        print("pred",predictions[i],"real",y_test[i])
        print(data_test[i])
        

pred 5 real 2
(결론,NNG) (만,JX) (이야기,NNG) (드리,VV) (ㅂ니다,EFN) (듣,VV) (지,ECD) (마,NNG) (세요,NNP) (로,JKM) (드,VV) (는,ETD) (21,NR) (학점,NNG) (정도,NNG) (되,VV) (ㅂ니다,EFN) (하루,NNG) (에,JKM) (3,NR) (시간,NNM) (씩,MAG) (코딩,NNG) (하,XSV) (어도,ECD) (힘들,VA) (ㄴ,ETD) (과제,NNG) (라고,JX) (교수님,NNG) (께,JKM) (말씀드리,VV) (었,EPT) (더니,ECD) (왜,MAG) (하루,NNG) (에,JKM) (3,NR) (시간,NNM) (밖에,JX) (안하,NNG) (이,VCP) (냐,EFQ) (고,JKQ) (답변,NNG) (을,JKO) (주,VV) (시,EPH) (네요,EFN) (그렇,VA) (다고,EFN) (성적,NNG) (을,JKO) (잘,MAG) (주,VV) (시,EPH) (는,ETD) (것,NNB) (도,JX) (아니,VCN) (ㅂ니다,EFN) (자신,NNG) (의,JKG) (한계,NNG) (를,JKO) (시험,NNG) (하,XSV) (고,ECE) (싶,VXA) (거나,ECE) (,,SP) (너무,MAG) (하,VV) (ㄹ,ETD) (것,NNB) (이,JKS) (없,VA) (고,ECE) (심심,XR) (하,XSA) (시,EPH) (ㄴ,ETD) (졸업,NNG) (직전,NNG) (이,VCP) (ㄴ,ETD) (분,NNG) (들,XSN) (은,JX) (추천,NNG) (드리,VV) (ㅂ니다,EFN)

pred 5 real 4
(Careful,OL) (with,OL) (your,OL) (homeworks,OL) (.,SF) (Go,OL) (to,OL) (labs,OL) (,,SP) (even,OL) (HW,OL) (introduction,OL) (labs,OL) (.,SF)

pred 5 real 4
(안,MAG) (좋,VA) (다는,ETD) (후기,NNG) (가,JKS) (압도적,NNG) (